# LangChain从入门到实践：构建智能应用的基础框架

LangChain是当前大模型应用开发中最流行的框架之一，它为开发者提供了一套完整的工具链，帮助我们更便捷地构建基于大模型的应用。本文将带领读者入门LangChain，掌握其核心概念和基础组件。

## 1. LangChain简介

LangChain是一个面向大模型的开发框架，旨在简化大模型应用的开发过程。它的核心优势在于：
统一了不同模型的调用接口
提供了丰富的组件和工具链
支持链式处理流程
便于集成外部知识和工具

LangChain的核心组件包括：

模型I/O封装：Chat Models、PromptTemplate、OutputParser等
数据连接封装：Document Loaders、Document Transformers、Text Embedding Models、Vectorstores & Retrievers等
架构封装：Chain/LCEL、Agent、Tools、LangGraph、LangSmith等

## 2. 环境准备

在开始前，让我们先安装必要的依赖：

In [1]:
# 安装LangChain和OpenAI包
%pip install langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


接下来创建.env文件，配置OpenAI API密钥：

OPENAI_API_KEY=your_api_key_here

## 3. 模型I/O封装

### 3.1 模型接口封装

LangChain对各种大模型提供了统一的接口，使我们能够轻松切换不同的模型而不用大幅修改代码。

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

# 加载环境变量
_ = load_dotenv(find_dotenv())

# 检查必要的环境变量是否存在
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("请在.env文件中设置OPENAI_API_KEY")

# 初始化模型
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o-mini", model_provider="openai")

response = model.invoke("你是谁")
print(response.content)

我是一个人工智能助手，旨在回答问题、提供信息和协助解决问题。如果你有任何问题，随时可以问我！


### 3.2 多轮对话封装

LangChain提供了标准化的消息类型，用于处理多轮对话：

In [3]:
from langchain.schema import (
    AIMessage,  # 等价于OpenAI接口中的assistant role
    HumanMessage,  # 等价于OpenAI接口中的user role
    SystemMessage  # 等价于OpenAI接口中的system role
)

messages = [
    SystemMessage(content="你是云廷科技的课程助理。"),
    HumanMessage(content="我是学员，我叫kleinSpero。"),
    AIMessage(content="欢迎！"),
    HumanMessage(content="我是谁")
]

ret = model.invoke(messages)
print(ret.content)

你是学员 kleinSpero。如果你有任何问题或者需要帮助，随时可以告诉我！


### 3.3 更换模型提供商

LangChain的一大优势是可以轻松切换不同的模型提供商。

以DeepSeek模型为例：

In [4]:
# 安装DeepSeek支持
# %pip install langchain-deepseek

# 使用DeepSeek模型
from langchain.chat_models import init_chat_model
import os
from dotenv import load_dotenv, find_dotenv

# 加载环境变量
_ = load_dotenv(find_dotenv())

# 检查必要的环境变量是否存在
if not os.getenv("DEEPSEEK_API_KEY"):
    raise ValueError("请在.env文件中设置DEEPSEEK_API_KEY")

model = init_chat_model("deepseek-chat", model_provider="deepseek")

response = model.invoke("你是谁")
print(response.content)

Note: you may need to restart the kernel to use updated packages.
我是DeepSeek Chat，由深度求索公司创造的智能助手！✨ 我的使命是帮助你解答各种问题，无论是学习、工作，还是日常生活中的小困惑。我可以陪你聊天、提供建议、查找资料，甚至帮你分析文件内容。  

有什么我可以帮你的吗？😊


以Qwen为例:

In [10]:
# 使用千问模型
from langchain.chat_models import init_chat_model
from langchain_community.llms import Tongyi
import os
from dotenv import load_dotenv, find_dotenv

# 加载环境变量
_ = load_dotenv(find_dotenv())

# 检查必要的环境变量是否存在
if not os.getenv("DASHSCOPE_API_KEY"):
    raise ValueError("请在.env文件中设置DASHSCOPE_API_KEY")

# 加载 Tongyi 模型
model = Tongyi(model_name="qwen-turbo", dashscope_api_key=os.getenv("DASHSCOPE_API_KEY"))  # 使用通义千问qwen-turbo模型

response = model.invoke("你是谁")
print(response)

我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我能够回答问题、创作文字，如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我在多国语言上都有很好的掌握，能为你提供多样化的帮助。有什么我可以帮到你的吗？


## 3.4 流式输出

对于长回答，流式输出可以提升用户体验：

In [ ]:
# openai、deepseek
for token in model.stream("你是谁"):
    print(token.content, end="")

In [ ]:
# 通义千问
for token in model.stream("你是谁"):
    print(token, end="")

我是通义千问，阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我能够回答问题、创作文字，如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程等任务。如果你有任何问题或需要帮助，欢迎随时告诉我！

## 4. Prompt模板

### 4.1 基础Prompt模板

Prompt模板可以帮助我们统一管理提示词，并支持变量替换：

In [13]:
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("给我讲个关于{subject}的笑话")
print("===Template===")
print(template)
print("===Prompt===")
print(template.format(subject='小明'))

# 使用模板调用LLM
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")
ret = llm.invoke(template.format(subject='小明'))
print(ret.content)

===Template===
input_variables=['subject'] input_types={} partial_variables={} template='给我讲个关于{subject}的笑话'
===Prompt===
给我讲个关于小明的笑话
小明和他的朋友们一起去郊游，大家都带了好吃的食物。小明带了一筐苹果。

大家坐在一起，朋友们开始分享自己的美食。小明一边吃苹果，一边看着朋友们的美味佳肴，心里有点羡慕。

他就突然说：“我觉得我的苹果也很好吃！”

大家好奇地问：“那你为什么不分享呢？”

小明笑着回答：“因为…我怕我的苹果会被偷！这可是我的秘密武器！”

朋友们疑惑地问：“什么秘密武器？”

小明得意地说：“因为这是我妈特意为我选的，因为她说苹果可以让人聪明！所以我只想自己偷偷享用！”

朋友们哈哈大笑，说：“那我们就祝你的秘密武器能让你变得更聪明吧！别忘了也要和我们分享哦！”


### 4.2 ChatPromptTemplate

ChatPromptTemplate用于构建对话式的提示模板：

In [14]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "你是{product}的客服助手。你的名字叫{name}"),
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

prompt = template.format_messages(
    product="AI大模型应用开发",
    name="小程",
    query="你是谁"
)

print(prompt)
ret = llm.invoke(prompt)
print(ret.content)

[SystemMessage(content='你是AI大模型应用开发的客服助手。你的名字叫小程', additional_kwargs={}, response_metadata={}), HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={})]
你可以叫我小程，我是AI大模型应用开发的客服助手。有什么我可以帮助你的吗？


### 4.3 MessagesPlaceholder

MessagesPlaceholder用于在模板中预留位置，以便后续填充历史消息：

In [15]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "Translate your answer to {language}."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder("history"), human_message_template]
)

from langchain_core.messages import AIMessage, HumanMessage
# 创建示例对话
human_message = HumanMessage(content="Who is Elon Musk?")
ai_message = AIMessage(
    content="Elon Musk is a billionaire entrepreneur, inventor, and industrial designer"
)
# 使用模板格式化消息
messages = chat_prompt.format_prompt(
    history=[human_message, ai_message], language="中文"
)
# 打印格式化后的消息
print(messages.to_messages())

result = llm.invoke(messages) 
print(result.content)

[HumanMessage(content='Who is Elon Musk?', additional_kwargs={}, response_metadata={}), AIMessage(content='Elon Musk is a billionaire entrepreneur, inventor, and industrial designer', additional_kwargs={}, response_metadata={}), HumanMessage(content='Translate your answer to 中文.', additional_kwargs={}, response_metadata={})]
埃隆·马斯克（Elon Musk）是一位亿万富翁企业家、发明家和工业设计师。


### 4.4 从文件加载Prompt模板

对于复杂的Prompt，可以从文件加载：

In [19]:
from langchain.prompts import PromptTemplate

# 假设文件内容为: 举一个关于{topic}的例子
template = PromptTemplate.from_file("./data/prompt/example_prompt_template.txt")
print("===Template===")
print(template)
print("===Prompt===")
print(template.format(topic='黑色幽默'))

===Template===
input_variables=['topic'] input_types={} partial_variables={} template='举一个关于{topic}的例子'
===Prompt===
举一个关于黑色幽默的例子


## 5. 结构化输出

### 5.1 使用Pydantic对象

LangChain可以直接输出Pydantic对象，使API交互更加便捷：

In [20]:
from pydantic import BaseModel, Field

# 定义输出对象结构
class Date(BaseModel):
    year: int = Field(description="Year")
    month: int = Field(description="Month")
    day: int = Field(description="Day")
    era: str = Field(description="BC or AD")

from langchain.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# 定义结构化输出的模型
structured_llm = llm.with_structured_output(Date)

template = """提取用户输入中的日期。
用户输入:
{query}"""

prompt = PromptTemplate(
    template=template,
)

query = "2023年四月6日天气晴..."
input_prompt = prompt.format_prompt(query=query)

result = structured_llm.invoke(input_prompt)
print(result)

year=2023 month=4 day=6 era='AD'


### 5.2 输出JSON格式

也可以使用JSON Schema定义输出格式：

In [21]:
json_schema = {
    "title": "Date",
    "description": "Formated date expression",
    "type": "object",
    "properties": {
        "year": {
            "type": "integer",
            "description": "year, YYYY",
        },
        "month": {
            "type": "integer",
            "description": "month, MM",
        },
        "day": {
            "type": "integer",
            "description": "day, DD",
        },
        "era": {
            "type": "string",
            "description": "BC or AD",
        },
    },
}
structured_llm = llm.with_structured_output(json_schema)

result = structured_llm.invoke(input_prompt)
print(result)

{'year': 2023, 'month': 4, 'day': 6, 'era': 'AD'}


### 5.3 使用OutputParser

OutputParser可以解析大模型的输出为指定格式：

In [22]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser(pydantic_object=Date)

prompt = PromptTemplate(
    template="提取用户输入中的日期。\n用户输入:{query}\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

input_prompt = prompt.format_prompt(query=query)
output = llm.invoke(input_prompt)
print("原始输出:\n"+output.content)

print("\n解析后:")
print(parser.invoke(output))

原始输出:
```json
{"year": 2023, "month": 4, "day": 6, "era": "AD"}
```

解析后:
{'year': 2023, 'month': 4, 'day': 6, 'era': 'AD'}


也可以使用PydanticOutputParser：

In [23]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Date)

input_prompt = prompt.format_prompt(query=query)
output = llm.invoke(input_prompt)
print("原始输出:\n"+output.content)

print("\n解析后:")
print(parser.invoke(output))

原始输出:
```json
{"year": 2023, "month": 4, "day": 6, "era": "AD"}
```

解析后:
year=2023 month=4 day=6 era='AD'


### 5.4 输出格式纠错

OutputFixingParser可以修复格式不正确的输出：

In [24]:
from langchain.output_parsers import OutputFixingParser
from langchain_openai import ChatOpenAI

# 创建修复解析器
new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())
# 创建错误数据并测试
bad_output = output.content.replace("4","四")
# 使用原始解析器尝试解析
print("PydanticOutputParser:")
try:
    parser.invoke(bad_output)
except Exception as e:
    print(e)
# 使用新的OutputFixingParser解析同样的错误数据
print("OutputFixingParser:")
print(new_parser.invoke(bad_output))

PydanticOutputParser:
Invalid json output: ```json
{"year": 2023, "month": 四, "day": 6, "era": "AD"}
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
OutputFixingParser:
year=2023 month=4 day=6 era='AD'


## 6. Function Calling

Function Calling是大模型的重要能力，LangChain提供了便捷的接口：

In [25]:
from langchain_core.tools import tool

# 使用@tool装饰器将普通函数转换为LangChain工具
@tool
def add(a: int, b: int) -> int:
    """Add two integers.  

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

# 绑定工具到模型
import json

llm_with_tools = llm.bind_tools([add, multiply])

query = "3的4倍是多少?"
messages = [HumanMessage(query)]

output = llm_with_tools.invoke(messages)
print(json.dumps(output.tool_calls, indent=4))

[
    {
        "name": "multiply",
        "args": {
            "a": 3,
            "b": 4
        },
        "id": "call_ineEpBk9521v5Bh7TXJDTu8c",
        "type": "tool_call"
    }
]


处理工具调用结果

In [26]:
# 执行工具调用
messages.append(output)

available_tools = {"add": add, "multiply": multiply}

for tool_call in output.tool_calls:
    selected_tool = available_tools[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

# 再次调用模型处理工具结果
result = llm_with_tools.invoke(messages)
print(result.content)

3的4倍是12。


总结

本文介绍了LangChain的基础概念和核心组件，包括模型封装、Prompt模板、结构化输出和Function Calling等。这些组件为我们构建大模型应用提供了强大的基础。
在后续文章中，我们将继续探索LangChain的高级特性，包括数据连接、向量检索、Chain构建和Agent开发等内容，敬请期待！

注意：本文代码使用了最新的LangChain API，如果您使用的是较早版本，可能需要进行相应调整。建议使用最新版本的LangChain以获得最佳体验